In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   




In [2]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_only_US_YAY.csv'
df_original = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df_original = df_original.drop('Unnamed: 0', 1)
df_original.shape



(1308727, 26)

In [4]:
df_original.head(5)

,folder_id,num_folder_members,folder_creation_date,date_last_change,user_id,email_domain,num_adds,num_edits,num_deletes,major_content_type,...,University_Name,Cleaned_university_name,Country,Geolocation,zip,lat,long,Rural_Urban_Continuum_Code,State_Name,Description
0,592732,4,2008-06-12,2012-12-25,77422,cornell.edu,NaN,NaN,NaN,office_docs,...,cornell university,cornell university,United States,"(42.4534492, -76.4735027) 14850",14850.0,42.453449,-76.473503,1.0,New York,Metropolitan Counties – In large metro area of...
1,592732,4,2008-06-12,2012-12-25,79630,cornell.edu,NaN,NaN,NaN,office_docs,...,cornell university,cornell university,United States,"(42.4534492, -76.4735027) 14850",14850.0,42.453449,-76.473503,1.0,New York,Metropolitan Counties – In large metro area of...
2,592732,4,2008-06-12,2012-12-25,77619,cornell.edu,80.0,72.0,0.0,office_docs,...,cornell university,cornell university,United States,"(42.4534492, -76.4735027) 14850",14850.0,42.453449,-76.473503,1.0,New York,Metropolitan Counties – In large metro area of...
3,604794,2,2008-06-27,2012-12-23,86572,unc.edu,21798.0,40498.0,10660.0,pdf,...,university of north carolina at chapel hill,university of north carolina at chapel hill,United States,"(35.9049122, -79.0469134)",NaN,35.904912,-79.046913,NaN,NaN,NaN
4,680152,2,2008-08-26,2013-01-04,76579,cs.princeton.edu,644.0,0.0,112.0,pdf,...,princeton university,princeton university,United States,"(40.3439888, -74.6514481) 08544",8544.0,40.343989,-74.651448,1.0,New Jersey,Metropolitan Counties – In large metro area of...


In [6]:
df_original.columns

Index([u'folder_id', u'num_folder_members', u'folder_creation_date',
       u'date_last_change', u'user_id', u'email_domain', u'num_adds',
       u'num_edits', u'num_deletes', u'major_content_type',
       u'major_content_ext', u'group_total_publ', u'group_num_papers_last',
       u'group_num_citations', u'folder_lifespan', u'simplified_domain',
       u'University_Name', u'Cleaned_university_name', u'Country',
       u'Geolocation', u'zip', u'lat', u'long', u'Rural_Urban_Continuum_Code',
       u'State_Name', u'Description'],
      dtype='object')

In [9]:

dict_tupla_index={}
dict_userID_aggregate_activity={}



for row in df_original.iterrows():

    lat=row[1]['lat']
    lon=row[1]['long']
    
    user_id= row[1]['user_id']
    key=user_id

    
    
    num_adds= row[1]['num_adds']
    num_edits= row[1]['num_edits']
    num_deletes= row[1]['num_deletes']


    num_publ = row[1]['group_total_publ']
    num_publ_last = row[1]['group_num_papers_last']
    num_cit = row[1]['group_num_citations']


    


    try:
        dict_userID_aggregate_activity[key]


    except KeyError:  # if it is the first time i encounter that geolocation

        dict_userID_aggregate_activity[key]={}  

        dict_userID_aggregate_activity[key]['University']=row[1]['Cleaned_university_name']
        dict_userID_aggregate_activity[key]['zip']=row[1]['zip']      
        dict_userID_aggregate_activity[key]['lat']=row[1]['lat']      
        dict_userID_aggregate_activity[key]['long']=row[1]['long']      
        dict_userID_aggregate_activity[key]['Rural_Urban_Continuum_Code']=row[1]['Rural_Urban_Continuum_Code']   
        dict_userID_aggregate_activity[key]['State']=row[1]['State_Name']   



        dict_userID_aggregate_activity[key]['num_adds']=[]           
        dict_userID_aggregate_activity[key]['num_edits']=[]           
        dict_userID_aggregate_activity[key]['num_deletes']=[]


        dict_userID_aggregate_activity[key]['common_num_publ']=[]           
        dict_userID_aggregate_activity[key]['common_num_papers_last']=[]            
        dict_userID_aggregate_activity[key]['common_num_citations']=[]



    dict_userID_aggregate_activity[key]['num_adds'].append(num_adds)                        
    dict_userID_aggregate_activity[key]['num_edits'].append(num_edits)        
    dict_userID_aggregate_activity[key]['num_deletes'].append(num_deletes)
    dict_userID_aggregate_activity[key]['common_num_publ'].append(num_publ)
    dict_userID_aggregate_activity[key]['common_num_papers_last'].append(num_publ_last)
    dict_userID_aggregate_activity[key]['common_num_citations'].append(num_cit)


print "done with the loop over rows"


        
dict_userID_aggregate_activity_NEW = dict_userID_aggregate_activity  # i make a copy so i dont have to re-run it

done with the loop over rows


In [23]:
#dict_userID_aggregate_activity = dict_userID_aggregate_activity_NEW

In [10]:
def most_common(lista):  # the list may have str and/or NANs
 
    #print "\n", lista
  
    try:
        new_lista = [str(x) for x in lista ]   # ojo!  str(np.nan)  = 'nan'
    except TypeError:        
        new_lista=str(lista)  # if it is not a list, but just one element
        
   # print new_lista
        
    new_lista2= [ x for x in new_lista if x !="nan" ]  # i remove the nans     
    #print new_lista2,len(new_lista2)
    #raw_input()
    if len(new_lista2)>0:
        most_common=max(set(new_lista2), key=new_lista2.count)
     #   print most_common
      #  raw_input()
        return most_common   # i pick the most common (str) element
    
    else:
        
        return np.nan

##################################
##################################





############  i sum up all activity
for key in dict_userID_aggregate_activity:

    dict_userID_aggregate_activity[key]['num_folders']=len(dict_userID_aggregate_activity[key]['num_adds'])

    
    dict_userID_aggregate_activity[key]['num_deletes']= np.nansum(dict_userID_aggregate_activity[key]['num_deletes'])  # sum ignoring nans
    dict_userID_aggregate_activity[key]['num_adds']= np.nansum(dict_userID_aggregate_activity[key]['num_adds'])
    dict_userID_aggregate_activity[key]['num_edits']= np.nansum(dict_userID_aggregate_activity[key]['num_edits'])


    dict_userID_aggregate_activity[key]['tot_activity']=  dict_userID_aggregate_activity[key]['num_deletes'] + dict_userID_aggregate_activity[key]['num_adds'] + dict_userID_aggregate_activity[key]['num_edits']

    try:
        dict_userID_aggregate_activity[key]['log_tot_activity']=math.log(dict_userID_aggregate_activity[key]['tot_activity'])
    except :

        dict_userID_aggregate_activity[key]['log_tot_activity']=0.

    #print "\n",key
    dict_userID_aggregate_activity[key]['common_num_publ']= most_common(dict_userID_aggregate_activity[key]['common_num_publ'])  # these are list of strings and nans
    dict_userID_aggregate_activity[key]['common_num_papers_last']= most_common(dict_userID_aggregate_activity[key]['common_num_papers_last'])
    dict_userID_aggregate_activity[key]['common_num_citations']= most_common(dict_userID_aggregate_activity[key]['common_num_citations'])


   

#     if dict_lat_log_aggregate_activity[key]['University']=='university of miami':
#         print dict_lat_log_aggregate_activity[key]
#         print 





############# i dump the dict so i dont have to calculate them every time
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/"

filename="dict_userID_aggregate_activity.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(dict_userID_aggregate_activity, f)
print "written:",path+filename


filename="dict_userID_df_index.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(dict_tupla_index, f)
print "written:",path+filename


written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_userID_aggregate_activity.pickle
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_userID_df_index.pickle


In [11]:
dict_userID_aggregate_activity 

{378535942: {'Rural_Urban_Continuum_Code': 1.0,
  'State': 'Delaware',
  'University': 'university of delaware',
  'common_num_citations': nan,
  'common_num_papers_last': nan,
  'common_num_publ': nan,
  'lat': 39.67795039999999,
  'log_tot_activity': 0.0,
  'long': -75.75061140000001,
  'num_adds': 0.0,
  'num_deletes': 0.0,
  'num_edits': 0.0,
  'num_folders': 1,
  'tot_activity': 0.0,
  'zip': 19716.0},
 150994952: {'Rural_Urban_Continuum_Code': 1.0,
  'State': 'Michigan',
  'University': 'university of michigan',
  'common_num_citations': nan,
  'common_num_papers_last': nan,
  'common_num_publ': nan,
  'lat': 42.2780436,
  'log_tot_activity': 0.0,
  'long': -83.7382241,
  'num_adds': 0.0,
  'num_deletes': 0.0,
  'num_edits': 0.0,
  'num_folders': 1,
  'tot_activity': 0.0,
  'zip': 48109.0},
 81788941: {'Rural_Urban_Continuum_Code': 1.0,
  'State': 'Ohio',
  'University': 'university of cincinnati',
  'common_num_citations': '1183.0-146545.0',
  'common_num_papers_last': '18.0-121

In [12]:
df_master_info_userID = pd.DataFrame.from_dict(dict_userID_aggregate_activity,orient='index')


In [13]:
df_master_info_userID.head()

,log_tot_activity,Rural_Urban_Continuum_Code,common_num_citations,zip,University,tot_activity,num_folders,long,common_num_publ,State,num_edits,common_num_papers_last,num_adds,lat,num_deletes
43604,11.658185,1.0,1.0-30.0,60208.0,northwestern university,115634.0,16,-87.675267,1.0-2.0,Illinois,14907.0,0.0-1.0,82291.0,42.056459,18436.0
43800,5.247024,1.0,NaN,10003.0,new york university,190.0,12,-73.996461,NaN,New York,0.0,NaN,190.0,40.729513,0.0
43987,12.420434,1.0,1183.0-146545.0,21218.0,johns hopkins university,247814.0,111,-76.620518,56.0-1350.0,Maryland,31641.0,18.0-1216.0,185911.0,39.329901,30262.0
45280,8.302018,1.0,NaN,53706.0,university of wisconsin-madison,4032.0,20,-89.412487,NaN,Wisconsin,336.0,NaN,2604.0,43.076592,1092.0
48267,4.025352,1.0,NaN,21218.0,johns hopkins university,56.0,2,-76.620518,NaN,Maryland,0.0,NaN,56.0,39.329901,0.0
